In [2]:
import sys
sys.path.insert(0, '..')
from models_neural.quote_attribution.classification_models import SourceQA, SourceClassifier
import json
try:
    import attrdict
except:
    ! pip install attrdict
    import attrdict
from models_neural.src.config_helper import TransformersConfig
import pandas as pd 
import zipfile
from models_neural.quote_attribution.utils_dataset import SourceQADataModule, SourceClassificationDataModule
import torch
from tqdm.auto import tqdm

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifactory.inf.bloomberg.com/artifactory/api/pypi/bloomberg-pypi/simple/, https://artifactory.inf.bloomberg.com/artifactory/api/pypi/python-dsp-wheels/simple


In [6]:
if remote: 
    # download pretrained models/spacy 
    ! aws s3 cp --recursive s3://aspangher/spacy/en_core_web_lg/ en_core_web_lg/   --endpoint http://s3.dev.obdc.bcs.bloomberg.com
    ! aws s3 cp s3://aspangher/transformer-pretrained-models/roberta-base-expanded-embeddings.zip .  --endpoint http://s3.dev.obdc.bcs.bloomberg.com
    with zipfile.ZipFile('roberta-base-expanded-embeddings.zip', 'r') as zip_ref:
        zip_ref.extractall()

download: s3://aspangher/spacy/en_core_web_lg/accuracy.json to en_core_web_lg/accuracy.json
download: s3://aspangher/spacy/en_core_web_lg/meta.json to en_core_web_lg/meta.json 
download: s3://aspangher/spacy/en_core_web_lg/ner/moves to en_core_web_lg/ner/moves 
download: s3://aspangher/spacy/en_core_web_lg/ner/cfg to en_core_web_lg/ner/cfg      
download: s3://aspangher/spacy/en_core_web_lg/parser/cfg to en_core_web_lg/parser/cfg
download: s3://aspangher/spacy/en_core_web_lg/parser/moves to en_core_web_lg/parser/moves
download: s3://aspangher/spacy/en_core_web_lg/tagger/cfg to en_core_web_lg/tagger/cfg 
download: s3://aspangher/spacy/en_core_web_lg/tagger/tag_map to en_core_web_lg/tagger/tag_map
download: s3://aspangher/spacy/en_core_web_lg/tokenizer to en_core_web_lg/tokenizer
download: s3://aspangher/spacy/en_core_web_lg/vocab/lookups_extra.bin to en_core_web_lg/vocab/lookups_extra.bin
download: s3://aspangher/spacy/en_core_web_lg/vocab/lookups.bin to en_core_web_lg/vocab/lookups.bin

In [8]:
remote = True
if remote:
    ! aws s3 cp s3://aspangher/source-exploration/data/our-annotated-data__stage-2.tsv  . --endpoint http://s3.dev.obdc.bcs.bloomberg.com
    data_fn = 'our-annotated-data__stage-2.tsv'
else:
    data_fn = '../models_neural/quote_attribution/data/our-annotated-data__stage-2.tsv'
data_df = pd.read_csv(data_fn, sep='\t', header=None)

download: s3://aspangher/source-exploration/data/our-annotated-data__stage-2.tsv to ./our-annotated-data__stage-2.tsv


In [ ]:
uploaded_trained_models = ! aws s3 ls s3://aspangher/source-exploration/./ --endpoint http://s3.dev.obdc.bcs.bloomberg.com
uploaded_trained_models = list(map(lambda x: ' '.join(x.split()[3:]), uploaded_trained_models))

# Sanity Check Classification Model

In [9]:
remote = True
if remote:
    ! aws s3 cp s3://aspangher/source-exploration/data/quote-attribution-classification__sanity-check-data.tsv  . --endpoint http://s3.dev.obdc.bcs.bloomberg.com
    data_fn = 'quote-attribution-classification__sanity-check-data.tsv'
else:
    data_fn = '../models_neural/quote_attribution/data/quote-attribution-classification__sanity-check-data.tsv'
data_df = pd.read_csv(data_fn, sep='\t', header=None)

download: s3://aspangher/source-exploration/data/quote-attribution-classification__sanity-check-data.tsv to ./quote-attribution-classification__sanity-check-data.tsv


In [33]:
model_name = 'Stage 2: Quote Attribution + Detection. Classification. Method 2. Our full dataset only. No train on None'

In [41]:
paths = list(filter(lambda x: model_name in x, uploaded_trained_models))
model_path = list(filter(lambda x: 'trial-' in x, paths))[0]
config_path = list(filter(lambda x: 'config-' in x, paths))[0]
if remote:
    # download config
    ! aws s3 cp "s3://aspangher/source-exploration/./$config_path"  . --endpoint http://s3.dev.obdc.bcs.bloomberg.com
                
    # download models
    ! aws s3 cp "s3://aspangher/source-exploration/./$model_path"  . --endpoint http://s3.dev.obdc.bcs.bloomberg.com

download: s3://aspangher/source-exploration/./config-Stage 2: Quote Attribution + Detection. Classification. Method 2. Our full dataset only. No train on None..json to ./config-Stage 2: Quote Attribution + Detection. Classification. Method 2. Our full dataset only. No train on None..json
download: s3://aspangher/source-exploration/./trial-Stage 2: Quote Attribution + Detection. Classification. Method 2. Our full dataset only. No train on None.__epoch=09-perplexity=0.00.ckpt to ./trial-Stage 2: Quote Attribution + Detection. Classification. Method 2. Our full dataset only. No train on None.__epoch=09-perplexity=0.00.ckpt


In [45]:
with open(config_path) as f:
    config_dict = json.load(f)

config_dict = attrdict.AttrDict(config_dict)
config_dict.pretrained_files_s3 = config_dict.pretrained_model_path
config_dict.model_type = 'roberta'

In [47]:
model = SourceClassifier.load_from_checkpoint(model_path, config=config_dict)
if remote:
    model = model.to('cuda')

../models_neural/src/layers_embeddings.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.default_max_pos = nn.Parameter(torch.tensor(self.max_position_embs - 1), requires_grad=False)


In [48]:
data_model = SourceClassificationDataModule(
    config=config_dict,
    data_fp=data_fn,
    model_type='roberta',
    max_length_seq=2048,
    pretrained_model_path=config_dict.pretrained_model_path,
    batch_size=1,
    spacy_path='en_core_web_lg'
)

In [ ]:
data_model.prepare_data()
data_model.setup()
dataloader = data_model.val_dataloader()

 26%|██▋       | 88/335 [02:07<04:47,  1.16s/it] 

# Classification Model

In [ ]:
model = 'config-Stage 2: Quote Attribution + Detection. Classification. Method 2. Our full dataset only. No train on None..json'

# QA Model

In [ ]:
qa_model_path = 'trial-Stage 2: Quote Attribution. Our dataset only.__epoch=05-perplexity=0.00.ckpt'
if remote:
    # download config
    ! aws s3 cp 's3://aspangher/source-exploration/./config-Stage 2: Quote Attribution. Our dataset only..json'  . --endpoint http://s3.dev.obdc.bcs.bloomberg.com
                
    # download models
    ! aws s3 cp "s3://aspangher/source-exploration/./$qa_model_path"  . --endpoint http://s3.dev.obdc.bcs.bloomberg.com

In [8]:
config_out = 'config-%s.json' % "Stage 2: Quote Attribution. Our dataset only."
with open(config_out) as f:
    config_dict = json.load(f)

config_dict = attrdict.AttrDict(config_dict)
config_dict.pretrained_files_s3 = config_dict.pretrained_model_path
config_dict.model_type = 'roberta'

In [9]:
model = SourceQA.load_from_checkpoint(model_path, config=config_dict)
if remote:
    model = model.to('cuda')

In [11]:
data_model = SourceQADataModule(
    config=config_dict,
    data_fp=data_fn,
    model_type='roberta',
    max_length_seq=2048,
    pretrained_model_path=config_dict.pretrained_model_path,
    batch_size=1,
    spacy_path='en_core_web_lg'
)

In [12]:
data_model.prepare_data()
data_model.setup()
dataloader = data_model.val_dataloader()

In [42]:
all_samples = []
for idx, sample in tqdm(enumerate(data_model.test_dataset), total=len(data_model.test_dataset)):
    sample = data_model.collate_fn([sample])

    with torch.no_grad():
        input_ids = sample['input_ids'].to('cuda')
        attention_mask=sample['attention_mask'].to('cuda')
        sentence_ids=sample['sentence_ids'].to('cuda')
        output = model.forward(input_ids=input_ids, attention_mask=attention_mask, sentence_ids=sentence_ids)

        start_logits, end_logits = output
        start, end = start_logits.argmax().to('cpu'), end_logits.argmax().to('cpu') 
    
    sample['pred_start'] = start
    sample['pred_end'] = end
    sample['start_logits'] = start_logits.to('cpu')
    sample['end_logits'] = end_logits.to('cpu')
    all_samples.append(sample)

100%|██████████| 712/712 [00:32<00:00, 22.18it/s]


In [92]:
samples_df = pd.DataFrame(all_samples)

In [96]:
# [['start_positions', 'end_positions', 'pred_start', 'pred_end']]
samples_df = (
    samples_df
        .assign(start_positions=lambda df: df['start_positions'].str.get(0).apply(int))
        .assign(end_positions=lambda df: df['end_positions'].str.get(0).apply(int)) 
        .assign(pred_start=lambda df: df['pred_start'].apply(int)) 
        .assign(pred_end=lambda df: df['pred_end'].apply(int)) 
)

In [115]:
samples_df = (samples_df
 .assign(true_span=lambda df: 
         df.apply(lambda x: x['input_ids'][:, x['start_positions']:x['end_positions']], axis=1)
           .apply(lambda x: data_model.tokenizer.decode(x[0]))
        )
 .assign(pred_span=lambda df: 
         df.apply(lambda x: x['input_ids'][:, x['pred_start']:x['pred_end']], axis=1)
           .apply(lambda x: data_model.tokenizer.decode(x[0]))
        )
)

In [123]:
t = samples_df[['true_span', 'pred_span']].applymap(lambda x: x.replace('</s><s>', ' '))

In [131]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [138]:
t.apply(lambda x: compute_f1(x['pred_span'], x['true_span']), axis=1).mean()

0.4470576309257075

In [139]:
t.apply(lambda x: compute_exact_match(x['pred_span'], x['true_span']), axis=1).mean()

0.43679775280898875

In [62]:
from sklearn.metrics import f1_score

In [87]:
start_end_df

,start_positions,end_positions,pred_start,pred_end
0,49,52,49,92
1,1387,1394,1387,22
2,1,3,27,335
3,235,239,240,239
4,277,279,157,159
...,...,...,...,...
707,363,365,857,365
708,635,640,1089,1093
709,97,101,97,101
710,614,618,295,777


In [ ]:
samples_df

In [65]:
start_end_df.pipe(lambda df: df['start_positions'] ==  df['pred_start']).mean()

0.5196629213483146

In [66]:
start_end_df.pipe(lambda df: df['end_positions'] ==  df['pred_end']).mean()

0.5084269662921348

In [84]:
def evaluate_overlap(row):
    temp_pred_start, temp_pred_end = row[['pred_start', 'pred_end']]
    pred_start = min(temp_pred_start, temp_pred_end)
    pred_end = max(temp_pred_start, temp_pred_end)
    
    true_start = row['start_positions']
    true_end = row['end_positions']
    
    start_mark = max(pred_start, true_start)
    end_mark = min(pred_end, true_end)
    
    num = max(0, end_mark - start_mark)
    denom = pred_end - pred_start
    
    return num / denom

In [86]:
start_end_df.apply(evaluate_overlap, axis=1).mean()

0.43925877688822845

In [115]:
from transformers import AutoTokenizer

In [117]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base-expanded-embeddings')

In [119]:
tokenizer.decode(sample['input_ids'][:, start:end][0])

' David W. Eaton'

In [136]:
start.to('cpu')

tensor(212)

In [123]:
end

tensor(216)

In [121]:
sample

{'start_positions': tensor([212]),
 'end_positions': tensor([216]),
 'input_ids': tensor([[    0, 15545,   661, 18718,   111,  2236,    96,     5,  2625,    81,
          25752,     9,   681,     8,  1123, 12252,  2156,  4257,   747, 22884,
              5,   810,    14,     5,   609,    64,   278,   160,  3027, 20396,
            479,     2,     0,  1708,  4211,   224,    14,    11,     5,   315,
            532,  2156, 25752,   111, 26914, 20396,    32,    45,  1537,   479,
              2,     0,  1121,   896,  2156,   959,  2156,    10, 22040,     9,
          20396,    11,  6055,   624,     5,   375,   292,   107,    34,    57,
           9702,     7, 25752,  2156,    50, 29215, 34905,  2156,    11,    61,
            514,  2156,  8321,     8,  6255,    32, 22993,    23,   239,  1164,
             88,    10,   157, 20463,    11,    10, 14352,  9285,     7,  1108,
             62,     5,  3152,     8,   800,   681,     8,  1123,   479,     2,
              0,  5975,  2156,  4211,  